[Back to Overview](overview.ipynb)

# Data Structures:  Keeping Data Organized
#### by Kirby Urner

The verb "keep" is interesting, in that if you've studied the anatomy of castles, you'll know the central building, where the crown, jewels and sceptre are kept, is called the Keep.

Keeping Data, might be a term of art, although it currently isn't.

Sometimes, in fairy tales, [a dragon](https://tvtropes.org/pmwiki/pmwiki.php/Main/DragonHoard) keeps guard over the hoard of treasure.

<a data-flickr-embed="true"  href="https://www.flickr.com/photos/kirbyurner/9583829639/in/photolist-27Qk7d8-XdQNYJ-VcSFee-oy7oP7-fATCE4-9oKHcV-9mV46z-8JMt7W-7uuc4g-7iy6jr-76Xswj-76XsFm-6WcupH-6QZD8h-6QVzKx-6y72Mo-6xXBCb-6xNgjY-5JVBmV-5z7Qey-5f9rAd-5f54mc" title="Browser as Theater"><img src="https://farm8.staticflickr.com/7301/9583829639_8199c0b3c6.jpg" width="500" height="375" alt="Browser as Theater"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Probably the first data structure you encounter in Python will be the list, although technically the string is a data structure (of letters, or str type objects).  

Then come the tuple (like an immutable list) and the dict (not even a sequence).  

All of these types use square brackets (`[]`) to retrieve contents, and matching up with square brackets are Python special names ```__getitem__``` and ```__setitem__```.

As Alex Martelli points out in one of his lectures, special names are deliberately "ugly" because we want to steer coders towards using square brackets when feasible.  Sometimes, though, we have to get behind the scenes, under the surface, into the guts of Python, and that's where special names (or ```__ribs___```) as I call them, come into focus.

In [1]:
the_tuple = (0, 2, 3, 'a', '🐙', '🐳', '🐯')
the_list = list(the_tuple[3:])
the_str = "".join(the_list)
the_dict = dict(zip(the_tuple[:-1], the_tuple[1:])) # pair each element with next in line

for thing in the_tuple, the_list, the_str, the_dict:
    print("thing of", type(thing), thing)

thing of <class 'tuple'> (0, 2, 3, 'a', '🐙', '🐳', '🐯')
thing of <class 'list'> ['a', '🐙', '🐳', '🐯']
thing of <class 'str'> a🐙🐳🐯
thing of <class 'dict'> {0: 2, 2: 3, 3: 'a', 'a': '🐙', '🐙': '🐳', '🐳': '🐯'}


The cell below demonstrates how ```__getitem__``` is the verb behind use of square brackets.  ```obj[n]``` translates to ```obj.__getitem__(n)``` in Python's mind (interpretation).  Likewise ```obj[n] = x``` is the same as saying ```obj.__setitem__(n) = x```.

In [2]:
for thing in the_tuple, the_list, the_str, the_dict:
    print(thing.__getitem__(0), end=", ")  # all four use __getitem__
    print(thing[0]) 

0, 0
a, a
a, a
2, 2


The function below returns pseudo-random strings of 1s and 0s, of the requested length.  We're going to use such strings to build a pseudo data set suitable for feeding to a Machine Learning algorithm.

In [3]:
from random import choice

def get_random_row(n):
    return "".join([str(choice([0,1])) for _ in range(n)])

for _ in range(5):
    sample = get_random_row(6)
    print(sample)

110010
001110
000000
010011
011000


## NumPy's ndarray

The next data structure we often encounter in a Pythonic project, if doing any kind of number crunching, is the star of numpy:  the n-dimensional array, or ndarray.

The cells below show ndarrays getting used in a typical fashion:  we feed them to a Machine Learning algorithm, a classifier.

Our example is deliberately meaningless, meaning we'll pick an arbitrary pattern of 1s and 0s to match with 1.

In [4]:
import numpy as np

def make_data(n):
    """
    Only the three patterns singled out below 
    Patterns: 1,2,3
    """
    X = np.empty(shape=(n,6), dtype=int)
    Y = np.empty(shape=(n,),  dtype=int)
    for row in range(n):
        features = get_random_row(6)
        y = 0  # unless a pattern matches
        
        # Patterns
        if features == "100001": # 1
            y = 1
        if features == "010010": # 2
            y = 1
        if features == "001100": # 3
            y = 1
            
        Y[row]    = y  # list of "right answers"
        # fancy list comprehension to convert strs to ints
        X[row, :] = list(map(lambda x: int(x), features))
    return X, Y
    
train_samples, train_answers = make_data(1000)

In [5]:
print(train_samples[0:5]); print(train_answers[0:5])

[[0 0 1 0 1 1]
 [1 1 1 1 1 1]
 [1 0 1 1 0 1]
 [0 0 0 1 1 0]
 [0 0 1 1 1 0]]
[0 0 0 0 0]


As a check, lets print the first 10 rows for which the corresponding ```train_answer``` is 1.  This is to be sure we understand which patterns match with 1, and to confirm the algorithm is working properly.

In [6]:
lines = 0
for idx, sample in enumerate(train_samples):
    if train_answers[idx]:
        print(sample)
        lines += 1
    if lines > 10:
        break

[0 1 0 0 1 0]
[1 0 0 0 0 1]
[0 1 0 0 1 0]
[1 0 0 0 0 1]
[0 1 0 0 1 0]
[0 1 0 0 1 0]
[0 0 1 1 0 0]
[0 1 0 0 1 0]
[0 0 1 1 0 0]
[0 0 1 1 0 0]
[0 1 0 0 1 0]


## Learning from Data

Now we're ready to take one of the Machine Learning algorithms down from the shelf.  Lets go with KNN and fit the training data so far.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_samples, train_answers) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [8]:
# data = [0., 1., 0., 0., 1., 0.]
data = [0, 0, 1, 1, 0, 0]
test = np.array(data, dtype=float)
test = test.reshape((1,6))
neigh.predict(test)

array([1])

The cell below is about validating that our model is sea-worthy.  We make new data, of course following the same pattern, and see if our model ```neigh``` is up to the job of recognizing for which patterns to predict a 1, all other times 0.

In [9]:
test_samples, test_answers = make_data(50)
neigh.score(test_samples, test_answers)

1.0

## Big Data

Little Data is very important i.e. "big" does not mean "more meaningful" it just means "more data" often a lot more.  When the data takes up a lot of space, then how it's managed, and how it's run through Machine Learning algorithms, becomes a topic in itself. Additional tools enter the picture, such as Apache Spark and Hadoop.